In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub

In [2]:
# Load in dataset
data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Filename","Date","Sensitivity","Document"]]

# Train / Test split
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)

In [3]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

In [4]:
text_input = [["Hello my name is Jack"],["My name is also Jack"]]

def long_encode(text_input):
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    total = np.array([])
    for chunk in text_input:
        chunk = list(chunk)
        encoder_inputs = preprocessing_layer(chunk)
        outputs = encoder(encoder_inputs)
        net = np.array(outputs['pooled_output'][0])
        total = np.append(total,net,axis=0)    
    return total

In [5]:
def get_split(text1):
  text1 = str(text1)
  l_total = []
  l_parcial = []
  if len(text1.split())//450 >0:
    n = len(text1.split())//450
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:500]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*450:w*450 + 500]
      l_total.append(" ".join(l_parcial))
  return l_total

train_x = pd.DataFrame(train_x)
train_x["Text Split"] = train_x["Document"].apply(get_split)
split_text = np.array(train_x["Text Split"])

In [6]:
def get_chunks(l):
    n = 500
    return [l[i:i + n] for i in range(0, len(l), n)] 

train_x = pd.DataFrame(train_x)
train_x["Text Split"] = train_x["Document"].apply(get_split)
split_text = train_x["Text Split"]

In [7]:
doc_number = 0
for document in split_text:
    doc_number += 1
    print("Encoding document number: " + str(doc_number))
    encoded = pd.DataFrame(long_encode(document))
    encoded.to_csv(str(doc_number) + str(".csv")

Encoding document number: 1
Encoding document number: 2
Encoding document number: 3
Encoding document number: 4
Encoding document number: 5
Encoding document number: 6
Encoding document number: 7
Encoding document number: 8
Encoding document number: 9
Encoding document number: 10


ResourceExhaustedError:  OOM when allocating tensor with shape[3384,12,128,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node StatefulPartitionedCall/StatefulPartitionedCall/bert_encoder/StatefulPartitionedCall/transformer/layer_11/self_attention/einsum/Einsum}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_restored_function_body_604215]

Function call stack:
restored_function_body
